In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the CNN model for spatial feature extraction (AMS-CNN)
def build_ams_cnn(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv1D(64, 3, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Conv1D(128, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Conv1D(256, 3, activation='relu', padding='same')(x)
    x = layers.GlobalMaxPooling1D()(x)
    return models.Model(inputs, x)

# Define LSTM model for temporal dependencies
def build_lstm(input_shape, num_layers=3, units=64):
    inputs = layers.Input(shape=input_shape)
    x = layers.LSTM(units, return_sequences=True)(inputs)
    for _ in range(num_layers - 1):
        x = layers.LSTM(units, return_sequences=True)(x)
    x = layers.LSTM(units)(x)
    return models.Model(inputs, x)

# Define the Transformer encoder model
def build_transformer(input_shape, num_heads=4, ff_dim=64):
    inputs = layers.Input(shape=input_shape)
    x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=ff_dim)(inputs, inputs)
    x = layers.GlobalAveragePooling1D()(x)
    return models.Model(inputs, x)

# Combined Model: CNN + LSTM + Transformer
def build_combined_model(input_shape, lstm_layers=3, lstm_units=64, transformer_heads=4, transformer_ff_dim=64):
    cnn = build_ams_cnn(input_shape)
    lstm = build_lstm(input_shape, lstm_layers, lstm_units)
    transformer = build_transformer(input_shape, transformer_heads, transformer_ff_dim)

    combined = layers.concatenate([cnn.output, lstm.output, transformer.output])
    x = layers.Dense(64, activation='relu')(combined)
    x = layers.Dense(1)(x)  # Output layer for regression
    model = models.Model(inputs=[cnn.input, lstm.input, transformer.input], outputs=x)
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model
